In [1]:
library("MetaCyto")
library(flowCore)
library(dplyr)
library(ggplot2)
library(tidyr)
library(FlowSOM)


Attaching package: ‘dplyr’

The following object is masked from ‘package:flowCore’:

    filter

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: igraph

Attaching package: ‘igraph’

The following object is masked from ‘package:tidyr’:

    crossing

The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union

The following objects are masked from ‘package:flowCore’:

    normalize, parent, tree

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



In [2]:
##### change cell markers #####
cytof_files = read.csv("result_05_cytof_files.csv")
marker_list = c('VIABILITY',  'TCRGD', 'IGD', 'HLADR', 'DNA2', 
                'DNA1', 'CELL_LENGTH', 'CD94', 'CD85J', 'CD8', 'CD56', 
                'CD45RA', 'CD4', 'CD38', 'CD33', 'CD3', 'CD28', 'CD27', 'CD25', 
                'CD24', 'CD20', 'CD19', 'CD161', 'CD16', 'CD14', 'CD127', 'CCR7')

In [3]:
# standarize cytof files
dir.create("standard_fcs")
for(i in 1:nrow(cytof_files)){
    f = read.FCS(file.path("CyTOF_data",cytof_files$name[i]))
    cn = MetaCyto::markerFinder(f)
    cn = gsub('DEAD','VIABILITY',cn)
    cn = gsub('EVENT_LENGTH','CELL_LENGTH',cn)

    expr = exprs(f)
    colnames(expr) = toupper(cn)
    if(length(setdiff(marker_list, colnames(expr)))==0){
        expr = expr[sample(size = 10000,x = 1:nrow(expr), replace = T),marker_list]
        #expr = apply(expr, 2, scale)
        expr = flowFrame(expr)
        write.FCS(expr,file.path("standard_fcs",cytof_files$name[i]))
    }

}


Warning message in dir.create("standard_fcs"):
“'standard_fcs' already exists”

In [4]:
# run flowSOM
fileName = list.files("standard_fcs", full.names = T)
fSOM <- FlowSOM(fileName,compensate = F,transform = TRUE,toTransform=c(1:length(marker_list)),
                  scale = TRUE,
                  colsToUse = c(1:length(marker_list)), xdim = 10, ydim = 10,
                  nClus = 20,seed = 42)

In [5]:
# write summary statistics from flowSOM 
meta_df = data.frame("metacluster"= fSOM$metaclustering, "cluster" = 1:100)
coreName = basename(fileName)

data_df = cbind.data.frame(fSOM$FlowSOM$data, cluster = fSOM$FlowSOM$map$mapping[,1],
                          fcs_files = rep(coreName, each = 10000) )
data_df = inner_join(data_df, meta_df, by = "cluster")
frac_df = data_df%>%group_by(fcs_files, metacluster)%>%summarise(fraction = n()/10000)
data_df = data_df %>% select(-cluster)%>%group_by(fcs_files, metacluster)%>%
    summarise_all(mean)

data_df = inner_join(data_df, frac_df, by = c("fcs_files", "metacluster"))
data_df = gather(data_df, value = "value", key = "type", -fcs_files, -metacluster)
data_df = data_df %>%mutate(CN = paste0(metacluster, type))%>%select(fcs_files, CN, value)%>%
    spread(key = CN, value = value)
data_df[,-1] = sapply(data_df[,-1], function(x){x[is.na(x)]=mean(x,na.rm = T); return(x)})
write.csv(data_df, "result_07_summary_statistics.csv", row.names= F)
data_df[1:50,]

fcs_files,10CCR7,10CD127,10CD14,10CD16,10CD161,10CD19,10CD20,10CD24,10CD25,⋯,9CD85J,9CD94,9CELL_LENGTH,9DNA1,9DNA2,9fraction,9HLADR,9IGD,9TCRGD,9VIABILITY
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
011514-Mike-Study 21-2011-plate 1-2100101_cells_found.455521.fcs,-0.37732265,-0.214761604,-0.14253903,-0.315389051,0.757945904,-0.2333583563,-0.33605555,0.084330089,-0.334468126,⋯,0.630355997,0.050396678,0.1446567,0.5030917,0.3772989,0.0049,0.022994683,-0.1098849529,6.267686,0.053635838
011514-Mike-Study 21-2011-plate 1-2100201_cells_found.455522.fcs,-0.10174923,0.222785129,0.33726775,-0.249876199,0.510515599,-0.0806139632,-0.19758586,0.118731761,-0.199927326,⋯,-0.102398080,1.395491461,0.6321092,0.5085522,0.3688550,0.0208,-0.196673810,-0.1733880842,6.069085,0.011073614
011514-Mike-Study 21-2011-plate 1-2100301_cells_found.455523.fcs,-0.09006003,0.028963587,0.50476025,-0.208113205,0.586457394,0.2481534458,-0.16038490,0.606628674,-0.080504452,⋯,0.408759271,0.360638880,0.5121311,0.3979136,0.2595999,0.0094,-0.158101160,-0.2090873606,5.220904,0.284268897
011514-Mike-Study 21-2011-plate 1-2100501_cells_found.455524.fcs,0.52710156,0.139389021,-0.19013565,-0.128891817,0.545304509,0.1031453138,-0.09260034,-0.107981847,-0.133605987,⋯,0.053530720,0.600946899,0.5298334,0.4940720,0.3872390,0.0068,-0.287350090,-0.1786796621,5.650374,0.863846944
011514-Mike-Study 21-2011-plate 1-2100601_cells_found.455525.fcs,-0.24258167,-0.118699408,0.03587966,-0.059393355,1.060393524,0.4969962169,-0.32703953,0.422035576,-0.225386294,⋯,-0.117435343,0.628599484,0.3566099,0.5113881,0.3986665,0.0090,-0.334772845,-0.1016586815,5.339016,0.635270112
011514-Mike-Study 21-2011-plate 1-2100701_cells_found.455526.fcs,-0.33603536,0.038666850,0.65474048,0.028449855,-0.313307989,-0.3349564629,-0.30672930,-0.093587058,0.032528844,⋯,-0.003676948,0.082883878,0.4408994,0.4975656,0.4035701,0.0067,-0.216229186,-0.2053684694,5.113605,0.548866883
011514-Mike-Study 21-2011-plate 1-2100901_cells_found.455527.fcs,-0.61533182,0.029364860,-0.31387349,0.007608105,1.254732339,-0.2916677569,-0.29351819,0.297092747,-0.471343409,⋯,-0.335597439,1.496056178,0.4615444,0.5363765,0.4053977,0.0110,-0.475663231,-0.2050757531,5.299119,0.425895568
011514-Mike-Study 21-2011-plate 1-2101001_cells_found.455528.fcs,0.79388050,0.097188043,-0.08358307,-0.037889583,1.192864305,-0.1568497818,-0.32883599,0.178516056,0.134644233,⋯,-0.258218251,0.335260954,0.7913592,0.5180540,0.4053977,0.0069,-0.358203174,-0.1990803427,5.270064,1.092871344
011514-Mike-Study 21-2011-plate 1-2101301_cells_found.455529.fcs,-0.34878779,-0.116986585,0.28694722,-0.205486618,0.268147709,-0.1025420692,0.16485885,0.763388449,-0.216313635,⋯,0.522621136,0.472943378,0.6710896,0.2755458,0.1788810,0.0041,0.414685253,-0.1474275774,5.179777,0.054927583
